In [3]:
# -*- coding: utf-8 -*-
"""
🎯 (2024+2025) 축제명 + 기준 월 → 같은 '계절(절기형)'에 열리는 축제만 리스트업 (유사시기 단일 필터)
- 계절 규칙: 3/4/5 → [3,4,5], 6/7/8 → [6,7,8], 9/10/11 → [9,10,11], 12/1/2 → [12,1,2]
- 로드: ./csv/{2024,2025}_축제_핵심필드.csv (+ /mnt/data 경로도 시도)
- 아직 시작 안 한 축제(시작일 > 오늘) 제외
- 입력 축제명(정확·유사) 제외
- 출력: 기본 "축제명  (표시월)" / show_period=True면 "축제명  시작일~종료일  (표시월)"
"""

import os, re
from pathlib import Path
from typing import Optional, List, Set, Dict

import pandas as pd
from difflib import get_close_matches

# ---------------- Settings --------------------
CANDIDATE_FILES = [
    "./csv/2024_축제_핵심필드.csv",
    "./csv/2025_축제_핵심필드.csv",
    "/mnt/data/2024_축제_핵심필드.csv",
    "/mnt/data/2025_축제_핵심필드.csv",
]

# ---------------- Utils -----------------------
def normalize_text(s: str) -> str:
    if s is None: return ""
    s = str(s).strip().replace("\xa0", " ")
    return re.sub(r"\s+", "", s)

def parse_date_str(s: str) -> Optional[pd.Timestamp]:
    try:
        return pd.to_datetime(str(s), format="%Y-%m-%d", errors="coerce")
    except Exception:
        return None

def today_floor_ts() -> pd.Timestamp:
    return pd.Timestamp.today().normalize()

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """'축제유형' -> '축제 유형' 등 표준화 (유형은 쓰지 않지만 스키마 정합성용)."""
    colmap = {}
    for c in df.columns:
        cc = str(c).strip()
        if cc == "축제유형":
            colmap[c] = "축제 유형"
    return df.rename(columns=colmap)

def ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = standardize_columns(df)
    needed = ["연번", "광역자치단체명", "기초자치단체명", "축제명", "시작일", "종료일"]
    for c in needed:
        if c not in df.columns:
            raise ValueError(f"필수 컬럼 누락: {c}")
    return df

def read_many_csv(paths: List[str]) -> pd.DataFrame:
    frames = []
    for p in paths:
        if Path(p).exists():
            try:
                df = pd.read_csv(p, dtype=str).fillna("")
            except UnicodeDecodeError:
                df = pd.read_csv(p, dtype=str, encoding="utf-8-sig").fillna("")
            df = ensure_columns(df)
            df["_출처파일"] = Path(p).name
            frames.append(df)
    if not frames:
        raise FileNotFoundError("입력 CSV를 하나도 찾지 못했습니다.")
    out = pd.concat(frames, ignore_index=True)
    out = out.drop_duplicates(
        subset=["광역자치단체명","기초자치단체명","축제명","시작일","종료일"],
        keep="first"
    ).reset_index(drop=True)
    return out

def months_in_range(start: Optional[pd.Timestamp], end: Optional[pd.Timestamp]) -> Set[int]:
    """시작~종료 사이의 월(1..12) 집합. 연도 넘어가도 월만 모음."""
    if (start is None or pd.isna(start)) or (end is None or pd.isna(end)):
        return set()
    if end < start:
        start, end = end, start
    months = set()
    cur = pd.Timestamp(year=start.year, month=start.month, day=1)
    last = pd.Timestamp(year=end.year, month=end.month, day=1)
    while cur <= last:
        months.add(int(cur.month))
        if cur.month == 12:
            cur = pd.Timestamp(year=cur.year + 1, month=1, day=1)
        else:
            cur = pd.Timestamp(year=cur.year, month=cur.month + 1, day=1)
    return months

def circular_month_distance(a: int, b: int) -> int:
    a, b = int(a), int(b)
    d = abs(a - b)
    return min(d, 12 - d)

# 계절 매핑: 3‒5 / 6‒8 / 9‒11 / 12‒2
def season_months(pivot_month: int) -> List[int]:
    m = int(pivot_month)
    if m in (3,4,5):   return [3,4,5]     # 봄
    if m in (6,7,8):   return [6,7,8]     # 여름
    if m in (9,10,11): return [9,10,11]   # 가을
    return [12,1,2]                       # 겨울 (연도 넘김)

# ---------------- Core ------------------------
def load_data(csv_path_override: Optional[str] = None) -> pd.DataFrame:
    if csv_path_override and Path(csv_path_override).exists():
        try:
            df = pd.read_csv(csv_path_override, dtype=str).fillna("")
        except UnicodeDecodeError:
            df = pd.read_csv(csv_path_override, dtype=str, encoding="utf-8-sig").fillna("")
        df = ensure_columns(df)
        df["_출처파일"] = Path(csv_path_override).name
        return df
    return read_many_csv(CANDIDATE_FILES)

def filter_by_season_time_only(
    df: pd.DataFrame,
    query_name: str,
    pivot_month: int,
) -> pd.DataFrame:
    """유사 시기(계절)만으로 필터링."""
    smonths = season_months(pivot_month)
    smonths_set = set(smonths)

    # 날짜 파싱 + 아직 시작 안 한 축제 제외
    today = today_floor_ts()
    df = df.copy()
    df["시작일_dt"] = df["시작일"].apply(parse_date_str)
    df["종료일_dt"] = df["종료일"].apply(parse_date_str)
    df = df[(~df["시작일_dt"].isna()) & (~df["종료일_dt"].isna()) & (df["시작일_dt"] <= today)]

    # 입력 축제명(정확·유사) 제외
    inorm = normalize_text(query_name)
    all_names = df["축제명"].astype(str).tolist()
    close = set(get_close_matches(query_name, all_names, n=3, cutoff=0.85))
    def _is_self(x: str) -> bool:
        nx = normalize_text(x)
        return (nx == inorm) or (x in close)
    df = df[~df["축제명"].astype(str).apply(_is_self)]

    # 계절과 겹치는 월이 있는 축제만 남김
    def _keep(row) -> bool:
        rng = months_in_range(row["시작일_dt"], row["종료일_dt"])
        return len(rng & smonths_set) > 0
    df = df[df.apply(_keep, axis=1)].copy()

    # 표시월: 계절 교집합 중 pivot에 가장 가까운 월 (원형 거리)
    df["시작월"] = df["시작일_dt"].dt.month.astype(int)
    def _display_month(row) -> int:
        rng = months_in_range(row["시작일_dt"], row["종료일_dt"]) & smonths_set
        if not rng:
            return int(row["시작월"])
        if pivot_month in rng:
            return int(pivot_month)
        return sorted(list(rng), key=lambda m: circular_month_distance(m, pivot_month))[0]
    df["표시월"] = df.apply(_display_month, axis=1).astype(int)

    # 계절 내부 자연스러운 순서(예: 겨울 12→1→2)로 정렬
    order_map = {m:i for i,m in enumerate(smonths, start=1)}
    df["_mo_order"] = df["표시월"].map(order_map).fillna(99)

    df = df.sort_values(by=["_mo_order","시작일_dt","축제명"]).drop(columns=["_mo_order"]).reset_index(drop=True)
    return df[["광역자치단체명","기초자치단체명","축제명","시작일","종료일","표시월","_출처파일"]]

# ---------------- Runner -----------------------
def run_timeonly_season(
    festival_name: str,
    pivot_month: int,
    csv_path_override: Optional[str] = None,
    print_limit: int = 300,
    dedupe: bool = True,
    show_period: bool = False,  # True면 "이름  시작~종료 (표시월)"
):
    df = load_data(csv_path_override)
    out = filter_by_season_time_only(df, festival_name, int(pivot_month))

    # 중복 이름 제거(원하면)
    if dedupe and not out.empty:
        out = out.drop_duplicates(subset=["축제명"], keep="first")

    # 출력
    print(f"입력 축제명: {festival_name} | 기준 월: {pivot_month} → 계절월={season_months(pivot_month)}")
    rows = []
    for _, r in out.iterrows():
        if show_period:
            rows.append(f"{r['축제명']}  {r['시작일']}~{r['종료일']}  ({int(r['표시월'])}월)")
        else:
            rows.append(f"{r['축제명']}  ({int(r['표시월'])}월)")

    print(f"\n같은 계절(유사시기) 축제 수: {len(rows)}")
    if rows:
        print("\n[목록]")
        for i, line in enumerate(rows[:print_limit], start=1):
            print(f"{i:>3}. {line}")
        if len(rows) > print_limit:
            print(f"... ({len(rows)-print_limit}개 더 있음)")
    return out

# ------------- Example -------------
if __name__ == "__main__":
    # 예: '담양산타축제', 12월 기준 → 겨울(12,1,2) 유사시기만
    run_timeonly_season("홍어축제", pivot_month=4, show_period=True)
    # run_timeonly_season("임실 산타축제", pivot_month=1, show_period=False)


입력 축제명: 홍어축제 | 기준 월: 4 → 계절월=[3, 4, 5]

같은 계절(유사시기) 축제 수: 498

[목록]
  1. 배내골고로쇠축제  2024-02-15~2024-03-15  (3월)
  2. 백련사 동백축제  2024-02-23~2024-03-03  (3월)
  3. 제52회 강진청자축제  2024-02-23~2024-03-03  (3월)
  4. 제27회 영덕대게축제  2024-02-29~2024-03-03  (3월)
  5. 제63회 3.1민속문화제  2024-02-29~2024-03-03  (3월)
  6. 동래 3.1만세운동재현행사  2024-03-01~2024-03-01  (3월)
  7. 논산딸기축제  2024-03-04~2024-03-04  (3월)
  8. 영흥풍어기원제  2024-03-04~2024-03-04  (3월)
  9. 제23회 광양매화축제  2024-03-08~2024-03-17  (3월)
 10. 동구 3.1절 기념 행사  2024-03-09~2024-03-09  (3월)
 11. 원동매화축제  2024-03-09~2024-03-17  (3월)
 12. 제17회 산수유마을꽃맞이행사  2024-03-09~2024-03-09  (3월)
 13. 제20회 진안고원 운장산 고로쇠 축제  2024-03-09~2024-03-10  (3월)
 14. 제25회 구례산수유꽃축제  2024-03-09~2024-03-17  (3월)
 15. 제44회 진도신비의바닷길축제  2024-03-11~2024-03-13  (3월)
 16. 연제고분판타지축제  2024-03-22~2024-03-24  (3월)
 17. 2024 대한민국 난명품 대제전  2024-03-23~2024-03-24  (3월)
 18. 대저토마토축제  2024-03-23~2024-03-24  (3월)
 19. 제32회 여수영취산진달래축제  2024-03-23~2024-03-24  (3월)
 20. 영암왕인문화축제  2024-03-28~2024-03-31  (3월)
 21. 